In [1]:
#用于1车道1交通灯和 pytorch测试
#作者lukeliuli@163.com
# -*- coding: utf-8 -*-
"""
@author: lukeliuli@163.com
"""
#%cd /root/myPrjs/SUMONBDT
import torch
import torchvision
import torchvision.transforms as transforms

import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

import time
import numpy as np
import os
import sys

import random
import csv
import datetime


def sortTakeSecond(elem):
    return elem[1] 



os.environ['SUMO_HOME'] = '/home/pi/myPrjs/sumo'
if 'SUMO_HOME' in os.environ:
     tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
     sys.path.append(tools)
else:
     sys.exit("please declare environment variable 'SUMO_HOME'")
    


import libsumo

path = os.getcwd()
path2 = path+"\sumoCfgs\my1Lane1TlsVeh3-server.sumocfg"#window
path2 = path+"/sumoCfgs1/my1Lane1TlsVeh3-server.sumocfg"#linux
sumoBinary = "sumo"
libsumo.start([sumoBinary, "-c", path2])


#定义模拟过程中动态可调整参数
params ={}
params["GreenTime"] = 15 #红灯时间
params["redTime"] = 25   #绿灯时间
params["ObjVehPosInXY"] = [-150,0] #主车位置
params["MaxAcc"] = 1              #最大加速度
params["DrvResTime"] = 0.5     # 驾驶员反应时间
params["addedVeh"] = 2+1        #停车线前面有多少车
params["accel"]=2.8     #最大加速度
params["tau"]=1  #驾驶员最大反应时间
    
numSim = 10000
nextTLSID = "2"
veh = "00003"
counterNoLowSpeed = 0
dataAllSim = []
for ii in range(numSim):
    print("###################################################################start sim %d" %ii)
    str1 = "./trainData/redTLS_"+str(ii)+"_sim.csv"
    csv_file = open(str1,"w+")    
    writer = csv.writer(csv_file)   



    libsumo.start([sumoBinary, "-c", path2])
    stepNum = 0
    requireStop = 0
    redLowSpeed = 0
    data = []
    ##修改出发位置，随机参数
    X =  params["ObjVehPosInXY"][0]+min(random.random()*150,80)#注意坐标的原点
    Y =  params["ObjVehPosInXY"][1]
    libsumo.vehicle.moveToXY(veh,"e1to2", 0, X, Y, -1073741824.0, 1)
    
    ##修改加速，反应时间
    vehOthers = "EBUS2"
    accel = params["accel"]
    accel = max(accel*random.random(),1)
    libsumo.vehicletype.setAccel(vehOthers,accel)
    
    tau = params["tau"]
    tau = max(tau*random.random(),0.3)
    libsumo.vehicletype.setTau(vehOthers,tau)
    
    ####修改绿灯时间，随机参数
    """
    greenTime =  params["GreenTime"]#注意坐标的原点
    greenTime = max(random.random()*greenTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    greenDurTime = lgc1.phases[1].duration
    yellowDurTime = lgc1.phases[2].duration
    redDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = greenTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
    """
    
    ####修改逻辑和红灯时间，随机参数
    redTime =  params["redTime"]#注意坐标的原点
    redTime = max(random.random()*redTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    
    lgc1.phases[1].state = 'r'
    redDurTime = lgc1.phases[1].duration
    
    yellowDurTime = lgc1.phases[2].duration
    
    lgc1.phases[3].state = 'G'
    greenDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = redTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
   
    #print(X,greenTime,greenDurTime,yellowDurTime,redDurTime)
    #print(lgc1)
    
    data = []
    
    ##先运行一步
    libsumo.simulationStep()
    stepNum += 1
    while requireStop == 0:
        libsumo.simulationStep()
        stepNum += 1
        
        ###交通灯设置
        ####https://sumo.dlr.de/daily/pydoc/traci._vehicle.html
        nextTLSID = "2"
        TLSPos = 0
        states = libsumo.trafficlight.getRedYellowGreenState(nextTLSID)
        timeT = libsumo.simulation.getCurrentTime()  # 当前时间
        nextSwitch = libsumo.trafficlight.getNextSwitch(nextTLSID)
        phaseName = libsumo.trafficlight.getPhaseName(nextTLSID)
        phase = libsumo.trafficlight.getPhase(nextTLSID)
        phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
        curTime = timeT/1000
        phaseLeftTime = nextSwitch - curTime
        
      
         

        #车设置
        veh = "00003"
        vehPos = libsumo.vehicle.getPosition(veh)[0]
        speed = libsumo.vehicle.getSpeed(veh)
        vehMaxSpeed = libsumo.vehicle.getMaxSpeed(veh)

        laneID = libsumo.vehicle.getLaneID(veh)
        #meanSpeed = libsumo.lane.getLastStepMeanSpeed(laneID)
        laneMaxSpeed = libsumo.lane.getMaxSpeed(laneID)

        vehMaxSpeed = min(vehMaxSpeed, laneMaxSpeed)
        
        dist = TLSPos-vehPos
        arrivalTime1 = min(100,dist / (speed + 0.001))
        arrivalTime2 = min(100,dist / vehMaxSpeed)
        #arrivalTime3 = min(100,dist / (meanSpeed+0.001))

        #predictMeanSpeed1min = laneMaxSpeed  #1分钟前绿灯时,当前场景下目标车道的平均速度
        #arrivalTime4 = min(dist / predictMeanSpeed1min,100)
        
        ##

       
        
        if states == "r" and  speed<10/3.6:
            print("low speed")
            redLowSpeed = 1
            requireStop = 1
            
        if states == "G" and  speed<10/3.6:
            print("low speed")
            redLowSpeed= 1
            requireStop = 1

      
            
            
        if timeT/1000 > 30 or dist<=0:  # if time is over 30 second stoping the simulation
            requireStop = 1
        #name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]    
        laneAVGSpeed = 40/3.6
        data1 =[phaseLeftTime,dist,speed,40/3.6,dist/(speed+0.01),dist/laneAVGSpeed]
        
        #前面只有停止的车，不存在运动的车,没有的车位置为0
        #vehTmp = "00001"
        #vehPos1 = libsumo.vehicle.getPosition(vehTmp)[0]
        #vehTmp = "00002"
        #vehPos2 = libsumo.vehicle.getPosition(vehTmp)[0]
        #name2 = ["红灯前车辆1位置","车辆2位置","车辆3位置","车辆4位置","车辆5位置","车辆6位置","车辆7位置","车辆8位置","速度一直高于1m/s"]    
        #laneAVGSpeed = 60/3.6
        #data2 =[TLSPos-vehPos1,TLSPos-vehPos2,0,0,0,0,0,0]
        #print("#####tls state: %s,duration:%.3f,running time(ms):%d,passTLS:%d,dist:%d,speed:%.2f" %(states,phaseDur,timeT,passTLSNoDeAcc,dist,speed))
        #data.append(data1+data2)
        
        
        ##获得车灯前的车.注意为了简单.最大为8辆车
        name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
        name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
       
        data2 =[0] * 16;
        vehicles = libsumo.vehicle.getIDList();  # 当前在道路上的车辆
        vehListTmp = []
        for vehTmp in vehicles:
            tlsList = libsumo.vehicle.getNextTLS(vehTmp)
            if vehTmp == "00003" or len(tlsList) == 0:
                continue
            tlsList = libsumo.vehicle.getNextTLS(vehTmp)
            tlsID_T, tlsIndex, distTmp, state = tlsList[0]
            speed = libsumo.vehicle.getSpeed(vehTmp)
            
            if tlsID_T == "2":
                vehListTmp.append([vehTmp,distTmp,speed]) 
                
        if len(vehListTmp)>0:
            vehListTmp.sort(key=sortTakeSecond)
            for i in range(len(vehListTmp)):
                data2[2*i]   = vehListTmp[i][1]
                data2[2*i+1] = vehListTmp[i][2]
                
        data.append(data1+data2)
                
             

                
       
    
    
    #####################################速度分段标记<10为0.10<= <20 为1 《=-20 《30 为2。30《 为3 
    speedFlag = -1
    veh = "00003"
    if redLowSpeed == 1:
        speedFlag = 0
        
    if redLowSpeed == 0:
        speed = libsumo.vehicle.getSpeed(veh)
        print("########################################speed:",speed)
        if speed>=10/3.6 and speed <20/3.6:
            speedFlag = 1
        if speed>=20/3.6 and speed <30/3.6:
            speedFlag = 2
        if speed>=30/3.6:
            speedFlag = 3
        print("############speedFlag:",speedFlag)
        print("############speedFlag:",speedFlag)
       
    
    ########################################
    time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    libsumo.close()
    print("###########################################################%s, sim %d close" %(time,ii))
    name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]   
    name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
    name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
       
    nameTmp = name1+name2+name3
    writer.writerow(nameTmp)
    #print(len(data))
    for i in range(len(data)):
        rowTmp=data[i]
        #print(type(rowTmp))
        rowTmp.extend([speedFlag])
        #print(rowTmp)
        writer.writerow(rowTmp)
        dataAllSim.append(rowTmp)
    csv_file.close()
    
    
   
    
    if redLowSpeed == 0:
        counterNoLowSpeed= counterNoLowSpeed+1
        
print("finish numSim:%d,LowSpeed:%d" %(numSim,numSim-counterNoLowSpeed))

#########################################################################################################
name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]   
name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
       
nameTmp = name1+name2+name3+["速度标志"] 
name1 = ["redLightTime","distToRedLight","speed","laneAvgSpeed","arriveTime1","arriveTime2"]   
name2 = ["vehPos_1","vehSpeed_1","vehPos_2","vehSpeed_2","vehPos_3","vehSpeed_3","vehPos_4","vehSpeed_4"] 
name3 = ["vehPos_5","vehSpeed_5","vehPos_6","vehSpeed_6","vehPos_7","vehSpeed_7","vehPos_8","vehSpeed_8"] 

headers = name1+name2+name3+["speedFlag"]
str1 = "./trainData/dataAllSim.csv"
with open(str1,'w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(dataAllSim)
#############################################################################################################    
!rm -rf ./trainData/redTLS*.csv
!zip -q  ./trainData/dataAllSim.zip ./trainData/dataAllSim.csv
#!rm -rf ./trainData/*.csv

###################################################################start sim 142
low speed
###########################################################2022-03-23 21:09:53, sim 142 close
###################################################################start sim 143
low speed
###########################################################2022-03-23 21:09:53, sim 143 close
###################################################################start sim 144
low speed
###########################################################2022-03-23 21:09:53, sim 144 close
###################################################################start sim 145
low speed
###########################################################2022-03-23 21:09:53, sim 145 close
###################################################################start sim 146
low speed
###########################################################2022-03-23 21:09:53, sim 146 close
###################################################################start si

###################################################################start sim 184
low speed
###########################################################2022-03-23 21:09:58, sim 184 close
###################################################################start sim 185
low speed
###########################################################2022-03-23 21:09:58, sim 185 close
###################################################################start sim 186
low speed
###########################################################2022-03-23 21:09:58, sim 186 close
###################################################################start sim 187
low speed
###########################################################2022-03-23 21:09:58, sim 187 close
###################################################################start sim 188
low speed
###########################################################2022-03-23 21:09:58, sim 188 close
###################################################################start si

low speed
###########################################################2022-03-23 21:10:03, sim 228 close
###################################################################start sim 229
low speed
###########################################################2022-03-23 21:10:03, sim 229 close
###################################################################start sim 230
low speed
###########################################################2022-03-23 21:10:03, sim 230 close
###################################################################start sim 231
low speed
###########################################################2022-03-23 21:10:04, sim 231 close
###################################################################start sim 232
low speed
###########################################################2022-03-23 21:10:04, sim 232 close
###################################################################start sim 233
########################################speed: 5.8821526188967335
#########

low speed
###########################################################2022-03-23 21:10:09, sim 270 close
###################################################################start sim 271
low speed
###########################################################2022-03-23 21:10:09, sim 271 close
###################################################################start sim 272
low speed
###########################################################2022-03-23 21:10:09, sim 272 close
###################################################################start sim 273
low speed
###########################################################2022-03-23 21:10:09, sim 273 close
###################################################################start sim 274
low speed
###########################################################2022-03-23 21:10:09, sim 274 close
###################################################################start sim 275
low speed
###########################################################2022-0

########################################speed: 5.533958839625893
############speedFlag: 1
############speedFlag: 1
###########################################################2022-03-23 21:10:14, sim 311 close
###################################################################start sim 312
low speed
###########################################################2022-03-23 21:10:14, sim 312 close
###################################################################start sim 313
low speed
###########################################################2022-03-23 21:10:14, sim 313 close
###################################################################start sim 314
low speed
###########################################################2022-03-23 21:10:14, sim 314 close
###################################################################start sim 315
low speed
###########################################################2022-03-23 21:10:15, sim 315 close
###################################################

...

low speed
###########################################################2022-03-23 21:31:45, sim 9805 close
###################################################################start sim 9806
low speed
###########################################################2022-03-23 21:31:45, sim 9806 close
###################################################################start sim 9807
low speed
###########################################################2022-03-23 21:31:45, sim 9807 close
###################################################################start sim 9808
low speed
###########################################################2022-03-23 21:31:45, sim 9808 close
###################################################################start sim 9809
low speed
###########################################################2022-03-23 21:31:46, sim 9809 close
###################################################################start sim 9810
low speed
#######################################################

low speed
###########################################################2022-03-23 21:31:51, sim 9847 close
###################################################################start sim 9848
low speed
###########################################################2022-03-23 21:31:51, sim 9848 close
###################################################################start sim 9849
low speed
###########################################################2022-03-23 21:31:51, sim 9849 close
###################################################################start sim 9850
low speed
###########################################################2022-03-23 21:31:51, sim 9850 close
###################################################################start sim 9851
low speed
###########################################################2022-03-23 21:31:51, sim 9851 close
###################################################################start sim 9852
low speed
#######################################################

########################################speed: 6.53446431513346
############speedFlag: 2
############speedFlag: 2
###########################################################2022-03-23 21:31:56, sim 9887 close
###################################################################start sim 9888
low speed
###########################################################2022-03-23 21:31:56, sim 9888 close
###################################################################start sim 9889
low speed
###########################################################2022-03-23 21:31:56, sim 9889 close
###################################################################start sim 9890
low speed
###########################################################2022-03-23 21:31:57, sim 9890 close
###################################################################start sim 9891
########################################speed: 5.862572656613889
############speedFlag: 2
############speedFlag: 2
#################################

low speed
###########################################################2022-03-23 21:32:02, sim 9926 close
###################################################################start sim 9927
low speed
###########################################################2022-03-23 21:32:02, sim 9927 close
###################################################################start sim 9928
low speed
###########################################################2022-03-23 21:32:02, sim 9928 close
###################################################################start sim 9929
low speed
###########################################################2022-03-23 21:32:02, sim 9929 close
###################################################################start sim 9930
low speed
###########################################################2022-03-23 21:32:02, sim 9930 close
###################################################################start sim 9931
low speed
#######################################################

low speed
###########################################################2022-03-23 21:32:07, sim 9968 close
###################################################################start sim 9969
########################################speed: 6.208583263978538
############speedFlag: 2
############speedFlag: 2
###########################################################2022-03-23 21:32:07, sim 9969 close
###################################################################start sim 9970
low speed
###########################################################2022-03-23 21:32:07, sim 9970 close
###################################################################start sim 9971
low speed
###########################################################2022-03-23 21:32:08, sim 9971 close
###################################################################start sim 9972
low speed
###########################################################2022-03-23 21:32:08, sim 9972 close
##########################################

In [ ]:
!rm -rf ./trainData/redTLS*.csv
!zip -q  ./trainData/dataAllSim.zip ./trainData/dataAllSim.csv
!rm -rf ./trainData/*.csv

In [ ]:
!rm -rf ./trainData/*.csv

In [ ]:
!git config --global user.email "lukeliuli@163.com"
!git config --global user.name "lukeliuli"
!git add .
!git commit -m  "tls OK, collectiong all and adding speedFlag"
!git push